In [1]:
import sys
sys.version

'3.10.4 (main, Jun 27 2022, 16:57:57) [GCC 11.2.0]'

# Stack frame

> A call stack is composed of stack frames (also called activation records or activation frames). These are machine dependent and ABI-dependent data structures containing subroutine state information. Each stack frame corresponds to a call to a subroutine which has not yet terminated with a return.

https://en.wikipedia.org/wiki/Call_stack#STACK-FRAME

Далее - согласно [ответу](https://stackoverflow.com/questions/40641615/what-is-the-difference-between-a-frame-and-object-and-when-should-i-modify-one).

In programming you have something called a `stack`. In Python, when you call a function you create something called a `stack frame`. This frame is (as you see in your example) basically just a table of all of the variables that are local to your function.

Note that defining a function doesn't create a new stack frame, it's the calling a function. For instance something like this:

In [2]:
def say_hello():
    name = input('What is your name?')
    print('Hello, {}'.format(name))

Your global frame is just going to hold one reference: `say_hello`. You can see that by checking out what's in the local namespace (in Python you pretty much have a 1:1 relationship between namespace, scope, and stack frames):

In [3]:
print(locals())

{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'import sys\nsys.version', "def say_hello():\n    name = input('What is your name?')\n    print('Hello, {}'.format(name))", 'print(locals())'], '_oh': {1: '3.10.4 (main, Jun 27 2022, 16:57:57) [GCC 11.2.0]'}, '_dh': [PosixPath('/home/victor/Dev/pythonista/function')], 'In': ['', 'import sys\nsys.version', "def say_hello():\n    name = input('What is your name?')\n    print('Hello, {}'.format(name))", 'print(locals())'], 'Out': {1: '3.10.4 (main, Jun 27 2022, 16:57:57) [GCC 11.2.0]'}, 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7f6a544bedd0>>, 'exit': <IPython.core.autocall.ZMQExitAutocall object at 0x7f6a544bf4c0>, 'quit': <IPython.core.autocall.ZMQExit

Note the dunder (short for double underscore double underscore) names - those are automagically provided, and for the purposes of our discussion you can ignore them. That leaves us with:

```
{'say_hello': <function say_hello at 0x101962d90>}
```

That 0x bit is the memory address where the function itself lives. So here, our global stack/frame contains just that one value. If you call your function and then check `locals()` again, you'll see that name isn't there. That's because when you call the function you create a new stack frame and the variable is assigned there. You can prove this by adding `print(locals())` at the end of your function. Then you'll see something like this:

```
{'name': 'Arthur, King of the Brits'}
```

No dunder names here. You'll also note that this doesn't show a memory address. If you want to know where this value lives, there's a function for that.

In [4]:
def say_hello():
    name = input('What is your name?')
    print('hello {}'.format(name))
    print(locals())
    print(id(name))
    return name

print(id(say_hello()))

hello Vic
{'name': 'Vic'}
140094586185904
140094586185904


# Objects

But what about objects? Well, in Python, everything is an object. Just try it: ...
They're all objects. But they may be different objects. And how can you tell? With `id`. Note that you also can compare whether or not two objects are the same object by using `is`.

In [5]:
x = 2512
y = 2512
x is y

False

In [6]:
2512 is (2500 + 12)  # в ответе было False

<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_82864/4134696615.py:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
  2512 is (2500 + 12)  # в ответе было False


True

In [7]:
x = 23
y = 23
x is y

True

In [8]:
23 is (20 + 3)

<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_82864/3745566927.py:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
  23 is (20 + 3)


True

Wait a minute, what happened there? Well, as it turns out, python (that is, CPython) [caches small integers](https://stackoverflow.com/questions/306313/is-operator-behaves-unexpectedly-with-integers).

One important thing to note is that the assignment operator = always assigns a new name to the same object. For example:

In [9]:
x = 592
y = 592
x is y, x == y  # False, True

(False, True)

In [10]:
x = y
x is y, x == y  # True, True

(True, True)

And it doesn't matter how many other names you give an object, or even if you pass the [object around to different frames](https://stackoverflow.com/questions/1132941/least-astonishment-and-the-mutable-default-argument), you still have the same object.